<a href="https://colab.research.google.com/github/HRL-Mike/MikesProject/blob/main/BLIP_GPT2_Gen_Endo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downlaod code and dataset

In [1]:
#Download code
!git clone https://github.com/mobarakol/Project_Gen.git
!mkdir /content/Project_Gen/datasets
%cd /content/Project_Gen/datasets

#Download Dataset
!gdown --id 1WGdztykX3nW6pi_BKp4rO8nA7ESNRfVN

# Unzipping the VQA EndoVis18 Dataset\
!unzip -q EndoVis-18-VQA.zip

%cd /content/Project_Gen

Cloning into 'Project_Gen'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 97 (delta 53), reused 25 (delta 8), pack-reused 0
Receiving objects: 100% (97/97), 43.32 KiB | 8.66 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/Project_Gen/datasets
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1WGdztykX3nW6pi_BKp4rO8nA7ESNRfVN
From (redirected): https://drive.google.com/uc?id=1WGdztykX3nW6pi_BKp4rO8nA7ESNRfVN&confirm=t&uuid=2ad88854-1556-41e6-a7bb-3cda289fdfb3
To: /content/Project_Gen/datasets/EndoVis-18-VQA.zip
100% 2.70G/2.70G [01:35<00:00, 28.3MB/s]
/content/Project_Gen


1. After download the Project_Gen folder, create a folder 'BLIP' under the Project_Gen folder. Then create two folders 'configs' and 'models' under the BLIP folder. Copy files from BLIP-configs and BLIP-models to the two created folders.
2. Modify the dataloader file 'dataloaderGPT2Sentence'. You should use transforms.Resize((224,224)) instead of (300,256).
self.transform = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
  ])
3. batch size=32 if GPU=24GB

Installing requirements

In [2]:
!pip install -q transformers

Running Script:<br>
%cd /content/Project_Gen
import os
os.makedirs('checkpoints/efvlegpt2rs18', exist_ok=True)
!python train_SGPT_V2_Sentence.py \
--lr=0.00001 \
--checkpoint_dir='checkpoints/efvlegpt2rs18/m18_v1_z_qf_' \
--dataset_type='m18' \
--tokenizer_ver='gpt2v1' \
--model_ver='efvlegpt2rs18' \
--model_subver='v1' \
--vis_pos_emb='zeroes'\
--batch_size=40

In [3]:
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.1 MB/s eta 0:00:00


In [4]:
!pip install -q fairscale

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
%cd /content/Project_Gen
import os
import sys
import argparse
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch import optim
from torch.nn import CrossEntropyLoss
from torch.utils.data  import DataLoader
from transformers import GPT2Tokenizer
import torch.backends.cudnn as cudnn
# from model.EFGPT2Sentence import EFVLEGPT2RS18Sentence
from dataloader.dataloaderGPT2Sentence import EndoVis18VQAGPTSentence
from utils import AverageMeter, save_clf_checkpoint, adjust_learning_rate

import torch.nn.functional as F
from torchvision import models
from transformers import VisualBertConfig, GPT2Config
from transformers import VisualBertModel, GPT2Model, ViTModel, SwinModel
from transformers import GPT2LMHeadModel
from nltk.translate.bleu_score import corpus_bleu

from BLIP.models.med import BertConfig, BertModel, BertLMHeadModel
from BLIP.models.blip import create_vit, init_tokenizer, load_checkpoint
from transformers import GPT2Config, GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

from PIL import Image
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


def seed_everything(seed=3407):
    '''
    Set random seed for reproducible experiments
    Inputs: seed number
    '''
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class BLIPGPTVQAGen(nn.Module):
    def __init__(self,
        med_config='/content/Project_Gen/BLIP/configs/med_config.json',  # 得用绝对路径
        num_class=18,
        image_size=224,
        vit='base',
        ):
        super().__init__()

        # visual encoder
        self.visual_encoder, vision_width = create_vit(vit, image_size, use_grad_checkpointing=False,
                                ckpt_layer=0, drop_path_rate=0.1)  # visual encoder
        # tokenizer
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token  # end of string

        # text encoder
        encoder_config = BertConfig.from_json_file(med_config)
        encoder_config.vocab_size = self.tokenizer.vocab_size  # 30524 --> 50257
        encoder_config.encoder_width = vision_width
        self.text_encoder = BertModel(config=encoder_config, add_pooling_layer=False)

        # decoder
        # change from BertLMHeadModel to GPT2Model
        self.gpt_decoder = GPT2LMHeadModel.from_pretrained('gpt2')

    def forward(self, image, question_inputs, answer_inputs):
        # visual encoder
        image_embeds = self.visual_encoder(image).to(device)
        image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image.device)

        # combine question and answer
        input_ids = torch.cat((question_inputs['input_ids'], answer_inputs['input_ids']), dim=1)
        attention_mask = torch.cat((question_inputs['attention_mask'], answer_inputs['attention_mask']), dim=1)

        # text encoder
        text_output = self.text_encoder(input_ids=input_ids.to(device),
                        attention_mask=attention_mask.to(device),
                        encoder_hidden_states=image_embeds.to(device),
                        encoder_attention_mask=image_atts.to(device),
                        return_dict=True)
        text_embeds = text_output.last_hidden_state  # torch.Size([1, 6, 768])

        # text decoder
        gpt_output = self.gpt_decoder(inputs_embeds=text_embeds.to(device),
                        encoder_attention_mask=attention_mask.to(device))

        return gpt_output

def train(args, train_dataloader, model, criterion, optimizer, epoch, tokenizer, device):

    model.train()

    total_loss = AverageMeter()

    for i, (_, visual_features, questions, answers) in enumerate(tqdm(train_dataloader),0):

        # prepare questions and answers
        question_list = []
        answer_list = []
        for question in questions: question_list.append(question)
        for answer in answers: answer_list.append(answer)

        if args.model_ver == 'efvlegpt2rs18' or args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
            question_inputs = tokenizer(question_list, padding="max_length",max_length= args.question_len, return_tensors="pt")
            answer_inputs = tokenizer(answer_list, padding="max_length",max_length= args.answer_len, return_tensors="pt")


        # Visual features
        visual_features = visual_features.to(device)
        visual_len = 80

        # model forward(question, img, answer)
        logits = model(visual_features, question_inputs, answer_inputs)[0]

        # only consider loss on reference summary just like seq2seq models
        idx = args.question_len + 1

        shift_logits = logits[..., idx:, :].contiguous()
        # print(f'shift_logits: {shift_logits.shape}') # torch.Size([40, 34, 50257])
        shift_labels = answer_inputs['input_ids'][..., 1:].contiguous() # 1 because answer has '<|sep|>' in front
        shift_labels = shift_labels.to(device)
        # print(f'shift_labels: {shift_labels.shape}') # shift_labels: torch.Size([40, 34])

        loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.update(loss.item())

    print("Epoch: {}/{} Loss: {:.6f} AVG_Loss: {:.6f}".format(epoch, args.epochs, total_loss.val, total_loss.avg))

def validate(args, val_loader, model, criterion, epoch, tokenizer, device, save_output = False):

    references = []
    hypotheses = []

    model.eval()
    total_loss = AverageMeter()

    with torch.no_grad():
        for i, (_, visual_features, questions, answers) in enumerate(tqdm(val_loader),0):

            # prepare questions and answers
            question_list = []
            answer_list = []
            for question in questions: question_list.append(question)
            for answer in answers: answer_list.append(answer)

            if args.model_ver == 'efvlegpt2rs18' or args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
                question_inputs = tokenizer(question_list, padding="max_length",max_length= args.question_len, return_tensors="pt")
                answer_inputs = tokenizer(answer_list, padding="max_length",max_length= args.answer_len, return_tensors="pt")


            # Visual features
            visual_features = visual_features.to(device)
            visual_len = 80

            # model forward(question, img, answer)
            logits = model(visual_features, question_inputs, answer_inputs)[0]

            # only consider loss on reference summary just like seq2seq models
            idx = args.question_len + 1
            shift_logits = logits[..., idx:, :].contiguous()
            shift_labels = answer_inputs['input_ids'][..., 1:].contiguous() # 1 because answer has '<|sep|>' in front

            # copy for logits and labels for sentence decoding and blue-4 score calculation
            logits_copy = logits.clone()
            shift_labels_copy = shift_labels.clone()

            # loss calculation
            shift_labels = shift_labels.to(device)
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            total_loss.update(loss.item())

            # references    - Ground truth answer
            answer_GT_dec = tokenizer.batch_decode(shift_labels_copy, skip_special_tokens= True)
            for answer_GT_dec_i in answer_GT_dec: references.append([answer_GT_dec_i.split()])
            # print(references)

            # Hypotheses - predicted answer
            _, answer_Gen_id = torch.max(logits_copy, dim=2)
            answer_Gen_dec = tokenizer.batch_decode(answer_Gen_id, skip_special_tokens= True)
            for answer_Gen_dec_i in answer_Gen_dec: hypotheses.append(answer_Gen_dec_i.split())
            # print(hypotheses)


        # Calculate BLEU1~4
        metrics = {}
        metrics["Bleu_1"] = corpus_bleu(references, hypotheses, weights=(1.00, 0.00, 0.00, 0.00))
        metrics["Bleu_2"] = corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0.00, 0.00))
        metrics["Bleu_3"] = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.00))
        metrics["Bleu_4"] = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))

        print("Epoch: {}/{} EVA LOSS: {:.6f} BLEU-1 {:.6f} BLEU2 {:.6f} BLEU3 {:.6f} BLEU-4 {:.6f}".format
          (epoch, args.epochs, total_loss.avg, metrics["Bleu_1"], metrics["Bleu_2"], metrics["Bleu_3"], metrics["Bleu_4"]))

    return metrics

def get_arg():
    parser = argparse.ArgumentParser(description='VisualQuestionAnswerClassification')

    # Training parameters
    parser.add_argument('--epochs',         type=int,   default=80,                                 help='number of epochs to train for (if early stopping is not triggered).') #80, 26
    parser.add_argument('--batch_size',     type=int,   default=40,                                 help='batch_size')
    parser.add_argument('--workers',        type=int,   default=1,                                  help='for data-loading; right now, only 1 works with h5pys.')

    # existing checkpoint
    parser.add_argument('--checkpoint',     default=None,                                           help='path to checkpoint, None if none.')

    parser.add_argument('--lr',             type=float, default=0.00001,                            help=' 0.00001, 0.000005')
    parser.add_argument('--checkpoint_dir', default= 'checkpoints/efvlegpt2rs18/m18/v3_p_qf_',      help='m18/c80')
    parser.add_argument('--dataset_type',   default= 'm18',                                         help='m18/c80')
    parser.add_argument('--tokenizer_ver',  default= 'gpt2v1',                                      help='btv2/btv3/gpt2v1')
    parser.add_argument('--model_subver',   default= 'v3',                                          help='V0,v1/v2/v3/v4')
    parser.add_argument('--question_len',   default= 25,                                            help='25')
    parser.add_argument('--answer_len',     default= 35,                                            help='25')
    parser.add_argument('--model_ver',      default= 'efvlegpt2rs18',                               help='efvlegpt2rs18/efvlegpt2Swin/"')  #vrvb/gpt2rs18/gpt2ViT/gpt2Swin/biogpt2rs18/vilgpt2vqa/efgpt2rs18gr/efvlegpt2Swingr
    parser.add_argument('--vis_pos_emb',    default= 'pos',                                         help='None, zeroes, pos')
    parser.add_argument('--patch_size',     default= 5,                                             help='1/2/3/4/5')

    parser.add_argument('--validate',       default=False,                                          help='When only validation required False/True')

    if 'ipykernel' in sys.modules:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

def blip_vqa_custom(pretrained='', **kwargs):
        model = BLIPGPTVQAGen(**kwargs)
        if pretrained:
            model, msg = load_checkpoint(model, pretrained)
        return model


if __name__ == '__main__':

    seed_everything()

    args = get_arg()
    args.lr = 0.00002 # 5e-5, 1e-5, 2e-5
    args.epochs = 80
    args.checkpoint_dir='checkpoints/m18_v1_z_qf_'
    args.dataset_type='m18'
    args.tokenizer_ver='gpt2v1'
    args.model_ver='efvlegpt2rs18'
    args.model_subver='v1'
    args.vis_pos_emb='zeroes'
    args.batch_size=32
    os.makedirs('checkpoints/efvlegpt2rs18', exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
    cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead
    print('device =', device)

    # best model initialize
    start_epoch = 1
    best_epoch = [0]
    best_results = [0.0]
    epochs_since_improvement = 0


    if args.dataset_type == 'm18':
        '''
        Train and test dataloader for EndoVis18
        '''
        # tokenizer
        tokenizer = None
        if args.tokenizer_ver == 'gpt2v1':
            tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer_length = len(tokenizer)

        # data location
        train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
        val_seq = [1, 5, 16]

        folder_head = 'datasets/EndoVis-18-VQA/seq_'
        folder_tail = '/vqa/Sentence/*.txt'

        # dataloader
        if args.model_ver == 'efvlegpt2rs18' or args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
            train_dataset = EndoVis18VQAGPTSentence(train_seq, folder_head, folder_tail, model_ver=args.model_ver)
            train_dataloader = DataLoader(dataset=train_dataset, batch_size= args.batch_size, shuffle=True, num_workers=8)
            val_dataset = EndoVis18VQAGPTSentence(val_seq, folder_head, folder_tail, model_ver=args.model_ver)
            val_dataloader = DataLoader(dataset=val_dataset, batch_size= args.batch_size, shuffle=False, num_workers=8)

    # Initialize / load checkpoint

    model_url = \
      'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'
    model = blip_vqa_custom(pretrained=model_url, image_size=224, vit='base')
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # Move to GPU, if available
    model = model.to(device)
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print('model params: ', pytorch_total_params)

    # Loss function
    criterion = CrossEntropyLoss().to(device)

    for epoch in range(start_epoch, args.epochs):

        if epochs_since_improvement > 0 and epochs_since_improvement % 5 == 0:
            adjust_learning_rate(optimizer, 0.8)

        # train
        train(args, train_dataloader=train_dataloader, model = model, criterion=criterion, optimizer=optimizer, epoch=epoch, tokenizer = tokenizer, device = device)

        # validation
        metrics = validate(args, val_loader=val_dataloader, model = model, criterion=criterion, epoch=epoch, tokenizer = tokenizer, device = device)

        if metrics["Bleu_4"] >= best_results[0]:
            epochs_since_improvement = 0

            best_results[0] = metrics["Bleu_4"]
            best_epoch[0] = epoch
            save_clf_checkpoint(args.checkpoint_dir, epoch, epochs_since_improvement, model, optimizer, best_results[0])
        else:
            epochs_since_improvement += 1
            print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))




/content/Project_Gen
device = cuda


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch: 8/80 Loss: 0.002006 AVG_Loss: 0.002245


100%|██████████| 101/101 [00:40<00:00,  2.49it/s]


Epoch: 8/80 EVA LOSS: 0.000002 BLEU-1 0.550194 BLEU2 0.530790 BLEU3 0.512022 BLEU-4 0.482630

Epochs since last improvement: 6



 11%|█         | 35/331 [00:39<05:22,  1.09s/it]Process Process-136:
Process Process-135:
Traceback (most recent call last):
Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7df1609b5b40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_ev

KeyboardInterrupt: 